In [3]:
import pandas as pd
import requests
import os
import json
from time import sleep

# Load links.csv
links_path = 'data/ml-32m/links.csv'
links = pd.read_csv(links_path)

# TMDb API setup
TMDB_API_KEY = "04826aaa5a7349d28828c140963b6483"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# Directory to save fetched data
output_dir = "data/enriched"
os.makedirs(output_dir, exist_ok=True)

# Function to fetch movie data from TMDb API
def fetch_movie_data(tmdb_id):
    url = f"{BASE_URL}{tmdb_id}"
    params = {"api_key": TMDB_API_KEY, "language": "en-US"}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for TMDb ID {tmdb_id}: {e}")
        return None

# Fetch and save data
for _, row in links.iterrows():
    tmdb_id = row['tmdbId']
    if pd.isna(tmdb_id):
        continue
    
    output_file = os.path.join(output_dir, f"{int(tmdb_id)}.json")
    if os.path.exists(output_file):
        print(f"Data for TMDb ID {tmdb_id} already exists. Skipping.")
        continue

    #print(f"Fetching data for TMDb ID {tmdb_id}...")
    movie_data = fetch_movie_data(int(tmdb_id))
    if movie_data:
        with open(output_file, 'w') as f:
            json.dump(movie_data, f)
    sleep(0.005)  # Avoid hitting rate limits


Data for TMDb ID 862.0 already exists. Skipping.
Data for TMDb ID 8844.0 already exists. Skipping.
Data for TMDb ID 15602.0 already exists. Skipping.
Data for TMDb ID 31357.0 already exists. Skipping.
Data for TMDb ID 11862.0 already exists. Skipping.
Data for TMDb ID 949.0 already exists. Skipping.
Data for TMDb ID 11860.0 already exists. Skipping.
Data for TMDb ID 45325.0 already exists. Skipping.
Data for TMDb ID 9091.0 already exists. Skipping.
Data for TMDb ID 710.0 already exists. Skipping.
Data for TMDb ID 9087.0 already exists. Skipping.
Data for TMDb ID 12110.0 already exists. Skipping.
Data for TMDb ID 21032.0 already exists. Skipping.
Data for TMDb ID 10858.0 already exists. Skipping.
Data for TMDb ID 1408.0 already exists. Skipping.
Data for TMDb ID 524.0 already exists. Skipping.
Data for TMDb ID 4584.0 already exists. Skipping.
Data for TMDb ID 5.0 already exists. Skipping.
Data for TMDb ID 9273.0 already exists. Skipping.
Data for TMDb ID 11517.0 already exists. Skipping

Parse the saved data and enrich the content features:

In [8]:
import os
import json
import pandas as pd
import csv

# Step 1: Read links.csv
links_df = pd.read_csv('data/ml-32m/links.csv')

# Directory containing the enriched JSON files
enriched_dir = 'data/enriched'

# Prepare a list to hold enriched movie data
enriched_data = []

# Step 2: Read all JSON files and extract the required fields
for file_name in os.listdir(enriched_dir):
    if file_name.endswith('.json'):
        tmdb_id = file_name.split('.')[0]

        # Read the JSON data
        with open(os.path.join(enriched_dir, file_name), 'r', encoding='utf-8') as f:
            movie_data = json.load(f)

        # Extract required fields
        title = movie_data.get("title", "")
        budget = movie_data.get("budget", None)
        imdb_id = movie_data.get("imdb_id", "")

        # origin_country from "production_countries" field
        production_countries = movie_data.get("production_countries", [])
        origin_country = ", ".join([c.get("name", "") for c in production_countries])

        original_language = movie_data.get("original_language", "")
        original_title = movie_data.get("original_title", "")
        overview = movie_data.get("overview", "")
        popularity = movie_data.get("popularity", None)
        release_date = movie_data.get("release_date", "")
        runtime = movie_data.get("runtime", None)
        tagline = movie_data.get("tagline", "")
        vote_average = movie_data.get("vote_average", None)
        vote_count = movie_data.get("vote_count", None)
        
        # Append extracted fields
        enriched_data.append({
            "tmdbId": tmdb_id,
            "title": title,
            "budget": budget,
            "imdb_id": imdb_id,
            "origin_country": origin_country,
            "original_language": original_language,
            "original_title": original_title,
            "overview": overview,
            "popularity": popularity,
            "release_date": release_date,
            "runtime": runtime,
            "tagline": tagline,
            "vote_average": vote_average,
            "vote_count": vote_count
        })

# Create a DataFrame from the enriched data
enriched_df = pd.DataFrame(enriched_data)

# Step 3: Escape text fields and handle line breaks
def escape_text_fields(df):
    text_columns = df.select_dtypes(include=['object']).columns
    for column in text_columns:
        # Escape line breaks and ensure proper quoting
        df[column] = df[column].apply(lambda x: str(x).replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)
    return df

# Escape text fields
enriched_df = escape_text_fields(enriched_df)

# Convert tmdbId to numeric if possible (links.csv often stores it as float)
enriched_df['tmdbId'] = pd.to_numeric(enriched_df['tmdbId'], errors='coerce')

# Step 4: Save the enriched data to movies_enriched.csv
output_file = "data/movies_enriched_32m.csv"
enriched_df.to_csv(output_file, index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
print(f"Enriched data saved to {output_file}")

Enriched data saved to data/movies_enriched_32m.csv


Check the created csv file:

In [9]:
import pandas as pd

# Load the CSV file
file_path = 'data/movies_enriched_32m.csv'  # Adjust to your actual file path
df = pd.read_csv(file_path, on_bad_lines='skip')

# Step 1: Check Numerical Columns for Invalid Values
numerical_columns = ['tmdbId', 'budget', 'runtime', 'vote_count', 'vote_average']

def check_numerical_columns(df, numerical_columns):
    invalid_values = {}
    for column in numerical_columns:
        # Try converting to numeric, invalid entries will become NaN
        converted = pd.to_numeric(df[column], errors='coerce')
        # Identify non-numeric entries (i.e., original values that became NaN)
        non_numeric_mask = converted.isna() & ~df[column].isna()
        non_numeric_values = df.loc[non_numeric_mask, column].unique()
        # Count non-numeric values
        non_numeric_count = len(non_numeric_values)
        invalid_values[column] = {
            'Original NaN': df[column].isna().sum(),
            'Non-Numeric Count': non_numeric_count,
            'Non-Numeric Values': non_numeric_values.tolist()
        }
    return invalid_values

numerical_issues = check_numerical_columns(df, numerical_columns)
print("Numerical Column Issues:")
for column, issues in numerical_issues.items():
    print(f"{column}: {issues}")

# Step 2: Impute Missing (NaN) Values with Column Mean
def impute_missing_values(df, numerical_columns):
    for column in numerical_columns:
        if column in df.columns:
            # Convert column to numeric, coercing invalid entries to NaN
            df[column] = pd.to_numeric(df[column], errors='coerce')
            # Replace NaN with the column mean
            mean_value = df[column].mean()
            df[column] = df[column].fillna(mean_value)
    return df

df = impute_missing_values(df, numerical_columns)
print("Missing values imputed with column means.")

# Step 3: Escape Non-Numerical Fields (Text) Appropriately
# def escape_text_fields(df):
#     text_columns = df.select_dtypes(include=['object']).columns
#     for column in text_columns:
#         df[column] = df[column].apply(lambda x: f'"{x}"' if isinstance(x, str) else x)
#     return df

# Escape text fields as per CSV standard
#df_escaped = escape_text_fields(df)

# Save the cleaned and escaped data to a new file
output_file = 'data/movies_enriched_32m_cleaned.csv'
df.to_csv(output_file, index=False, quoting=1)  # quoting=1 ensures all non-numeric fields are properly escaped
print(f"Cleaned and escaped data saved to: {output_file}")


Numerical Column Issues:
tmdbId: {'Original NaN': 0, 'Non-Numeric Count': 0, 'Non-Numeric Values': []}
budget: {'Original NaN': 0, 'Non-Numeric Count': 0, 'Non-Numeric Values': []}
runtime: {'Original NaN': 0, 'Non-Numeric Count': 0, 'Non-Numeric Values': []}
vote_count: {'Original NaN': 0, 'Non-Numeric Count': 0, 'Non-Numeric Values': []}
vote_average: {'Original NaN': 0, 'Non-Numeric Count': 0, 'Non-Numeric Values': []}
Missing values imputed with column means.
Cleaned and escaped data saved to: data/movies_enriched_32m_cleaned.csv


Now join the movies_enriched.csv with the movies.csv.

In [10]:
import os
import json
import pandas as pd

# Step 1: Read links.csv
links_df = pd.read_csv('data/ml-32m/links.csv')

# Step 4: Read the newly created movies_enriched.csv
movies_enriched_df = pd.read_csv("data/movies_enriched_32m_cleaned.csv")

# Step 5: Read movies.csv
movies_df = pd.read_csv("data/ml-32m/movies.csv")

# Step 6: Join movies.csv and movies_enriched.csv via movieId, tmdbId from links.csv

# Convert movieId in links to int if needed
links_df['movieId'] = pd.to_numeric(links_df['movieId'], errors='coerce')
links_df['tmdbId'] = pd.to_numeric(links_df['tmdbId'], errors='coerce')

# Merge links_df with movies_enriched_df on tmdbId first
links_enriched_merged = pd.merge(links_df, movies_enriched_df, on='tmdbId', how='left')

# Now merge the above result with movies_df on movieId
final_merged_df = pd.merge(movies_df, links_enriched_merged, on='movieId', how='left')

# Step 7: Save the final merged DataFrame
final_merged_df.to_csv("data/ml-32m/movies_with_enriched_data.csv", index=False)

print("Merging complete! The final file is: movies_with_enriched_data.csv")


Merging complete! The final file is: movies_with_enriched_data.csv
